In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

mdwoahidurrahman_figs_braintumor_segmentation_classification_sorted_path = kagglehub.dataset_download('mdwoahidurrahman/figs-braintumor-segmentation-classification-sorted')

print('Data source import complete.')


In [ ]:
import pandas as pd
import numpy as np, cv2, os, tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras import layers, Model, Input
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from tensorflow.keras.utils import Sequence
import albumentations as A
from tensorflow.keras.callbacks import ReduceLROnPlateau

2025-08-01 19:13:09.595657: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754075589.788165      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754075589.841242      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
def load_brisc_data(data_dir, split='train'):
    """
    Reads:
      <data_dir>/classification_task/{split}/{class}/*.jpg      (e.g. 2948.jpg)
      <data_dir>/segmentation_task/{split}/images/*.jpg         (e.g. glioma_2948.jpg)
      <data_dir>/segmentation_task/{split}/masks/*.png

    Returns a DataFrame with columns:
      image_path : full path to the segmentation image
      mask_path  : full path to the matching mask
      label      : integer 0=glioma,1=meningioma,2=pituitary
    """

    cls_root    = os.path.join(data_dir, 'classification_task', split)
    seg_img_dir = os.path.join(data_dir, 'segmentation_task',    split, 'images')
    seg_msk_dir = os.path.join(data_dir, 'segmentation_task',    split, 'masks')

    # your label map
    class_map = {'glioma':0, 'meningioma':1, 'pituitary':2}

    records = []
    # walk each class folder in classification_task
    for cls_name, cls_idx in class_map.items():
        cls_folder = os.path.join(cls_root, cls_name)
        if not os.path.isdir(cls_folder):
            continue

        for fname in os.listdir(cls_folder):
            if not fname.lower().endswith('.jpg'):
                continue

            # fname is e.g. "2948.jpg"
            img_id = os.path.splitext(fname)[0]

            # find the segmentation image that ends with _{img_id}.jpg
            matches = [f for f in os.listdir(seg_img_dir)
                       if f.endswith(f"_{img_id}.jpg")]
            if not matches:
                print(f"[!] no seg image for {fname}")
                continue

            seg_fname = matches[0]                    # e.g. "glioma_2948.jpg"
            mask_fname = seg_fname.replace('.jpg', '.png')
            img_path   = os.path.join(seg_img_dir, seg_fname)
            mask_path  = os.path.join(seg_msk_dir, mask_fname)

            if not os.path.exists(mask_path):
                print(f"[!] no mask for {seg_fname}")
                continue

            records.append({
                'image_path': img_path,
                'mask_path' : mask_path,
                'label'     : cls_idx
            })

    df = pd.DataFrame(records)
    if df.empty:
        raise RuntimeError(f"No data found under {data_dir!r} split={split}")
    return df

In [ ]:
# --- Data Generator (now reads full paths) ---
class MultiTaskDataGenerator(Sequence):
    def __init__(self, df, batch_size=8, img_size=(256,256)):
        """
        df : DataFrame with columns ['image_path','mask_path','label']
        """
        self.df = df.reset_index(drop=True)
        self.batch_size = batch_size
        self.img_size   = img_size

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def __getitem__(self, idx):
        batch = self.df.iloc[idx*self.batch_size : (idx+1)*self.batch_size]
        X, Y_seg, Y_cls = [], [], []

        for _, row in batch.iterrows():
            img  = cv2.imread(row['image_path'], cv2.IMREAD_GRAYSCALE)
            msk  = cv2.imread(row['mask_path'],  cv2.IMREAD_GRAYSCALE)
            if img is None or msk is None:
                continue

            img = cv2.resize(img, self.img_size) / 255.0
            msk = cv2.resize(msk, self.img_size) / 255.0

            # add channel dimension
            X.append(img.astype(np.float32)[...,None])
            Y_seg.append(msk.astype(np.float32)[...,None])
            Y_cls.append(row['label'])

        X     = np.stack(X, axis=0)
        Y_seg = np.stack(Y_seg, axis=0)
        Y_cls = tf.keras.utils.to_categorical(Y_cls, num_classes=3).astype(np.float32)

        return X, {'segmentation': Y_seg, 'classification': Y_cls}

In [ ]:
# --- Data Generator with Albumentations ---
class MultiTaskDataGenerator(Sequence):
    def __init__(self, df, batch_size=8, img_size=(256,256), augment=False):
        """
        df        : DataFrame with columns ['image_path','mask_path','label']
        augment   : whether to apply on-the-fly augmentation
        """
        self.df = df.reset_index(drop=True)
        self.batch_size = batch_size
        self.img_size   = img_size
        # define transform pipeline
        if augment:
            self.transform = A.Compose([
                A.HorizontalFlip(p=0.5),
                A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
                A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
                A.OneOf([
                    A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.3),
                    A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.3),
                    A.OpticalDistortion(distort_limit=0.3, p=0.3),
                ], p=0.3),
                A.Resize(img_size[0], img_size[1]),
            ], additional_targets={'mask': 'mask'})
        else:
            self.transform = A.Compose([
                A.Resize(img_size[0], img_size[1])
            ], additional_targets={'mask': 'mask'})

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def __getitem__(self, idx):
        batch = self.df.iloc[idx*self.batch_size : (idx+1)*self.batch_size]
        X, Y_seg, Y_cls = [], [], []

        for _, row in batch.iterrows():
            img = cv2.imread(row['image_path'], cv2.IMREAD_GRAYSCALE)
            msk = cv2.imread(row['mask_path'],  cv2.IMREAD_GRAYSCALE)
            if img is None or msk is None:
                continue

            # apply augmentation / resize
            augmented = self.transform(image=img, mask=msk)
            img, msk = augmented['image'], augmented['mask']

            # normalize to [0,1]
            img = img.astype(np.float32) / 255.0
            msk = msk.astype(np.float32) / 255.0

            # add channel dim
            X.append(img[..., None])
            Y_seg.append(msk[..., None])
            Y_cls.append(row['label'])

        X     = np.stack(X, axis=0)
        Y_seg = np.stack(Y_seg, axis=0)
        Y_cls = tf.keras.utils.to_categorical(Y_cls, num_classes=3).astype(np.float32)

        return X, {'segmentation': Y_seg, 'classification': Y_cls}

In [ ]:
# --- Dice Score and Loss ---
def dice_score(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

class DiceLoss(tf.keras.losses.Loss):
    def call(self, y_true, y_pred):
        return 1.0 - dice_score(y_true, y_pred)

bce = tf.keras.losses.BinaryCrossentropy()

def combined_dice_bce(y_true, y_pred):
    return bce(y_true, y_pred) + DiceLoss()(y_true, y_pred)

In [ ]:
# --- Model Definition (unchanged) ---
def conv_block(x, filters, kernel_size=3, strides=1):
    x = layers.Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x

def encoder_block(x, filters):
    x = conv_block(x, filters)
    x = conv_block(x, filters)
    p = layers.MaxPooling2D(pool_size=(2, 2))(x)
    return x, p

def decoder_block(x, skip, filters):
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Concatenate()([x, skip])
    x = conv_block(x, filters)
    x = conv_block(x, filters)
    return x

def cbam_module(x, reduction_ratio=8):
    channels = int(x.shape[-1])
    avg_pool = layers.GlobalAveragePooling2D()(x)
    max_pool = layers.GlobalMaxPooling2D()(x)
    shared_dense_1 = layers.Dense(channels // reduction_ratio, activation='relu')
    shared_dense_2 = layers.Dense(channels)
    mlp_avg = shared_dense_2(shared_dense_1(avg_pool))
    mlp_max = shared_dense_2(shared_dense_1(max_pool))
    channel_attention = layers.Add()([mlp_avg, mlp_max])
    channel_attention = layers.Activation('sigmoid')(channel_attention)
    channel_attention = layers.Reshape((1, 1, channels))(channel_attention)
    return layers.Multiply()([x, channel_attention])

def build_encoder(inputs):
    skips = []
    x = inputs
    for f in [32, 64, 128]:
        s, x = encoder_block(x, f)
        skips.append(s)
    x = conv_block(x, 256)
    return x, skips

def build_segmentation_decoder(x, skips):
    for i, f in reversed(list(enumerate([128, 64, 32]))):
        x = decoder_block(x, skips[i], f)
    x = layers.Conv2D(1, (1, 1), activation='sigmoid', name='segmentation')(x)
    return x

def build_classification_head(x, num_classes):
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(num_classes, activation='softmax', name='classification')(x)
    return x

def build_braintumnet(input_shape=(256, 256, 1), num_classes=3):
    inputs = Input(shape=input_shape)
    x, skips = build_encoder(inputs)
    x = cbam_module(x)
    seg_output = build_segmentation_decoder(x, skips)
    cls_output = build_classification_head(x, num_classes)
    return Model(inputs=inputs, outputs=[seg_output, cls_output])

In [ ]:
# === Cell 2: Load & split ===
if __name__ == '__main__':
    DATA_DIR = '/kaggle/input/figs-braintumor-segmentation-classification-sorted/restructured_dataset'
    df = load_brisc_data(DATA_DIR, split='train')
    train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

    train_gen = MultiTaskDataGenerator(train_df, batch_size=8, img_size=(256,256), augment = True)
    val_gen   = MultiTaskDataGenerator(val_df,   batch_size=8, img_size=(256,256), augment = False)

    model = build_braintumnet()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss={'segmentation': combined_dice_bce, 'classification': 'categorical_crossentropy'},
        loss_weights={'segmentation': 1.0, 'classification': 0.7},
        metrics={'segmentation': dice_score,   'classification': 'accuracy'}
    )

/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipykernel_36/4243090114.py:18: UserWarning: Argument(s) 'alpha_affine' are not valid for transform ElasticTransform
  A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.3),
I0000 00:00:1754075681.458563      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [ ]:
# === Cell 2: define your two checkpoints ===

from tensorflow.keras.callbacks import ModelCheckpoint

# 1) Best-model checkpoint (only when val segmentation dice_score improves)
checkpoint_best = ModelCheckpoint(
    'braintumnet_best.h5',
    monitor='val_segmentation_dice_score',
    mode='max',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

# 2) “Last” checkpoint (overwrite every epoch)
checkpoint_last = ModelCheckpoint(
    'braintumnet_last.h5',
    save_best_only=False,
    save_weights_only=False,
    verbose=1
)


reduce_lr = ReduceLROnPlateau(
    monitor='val_segmentation_dice_score',
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

callbacks = [checkpoint_best, checkpoint_last, reduce_lr]


In [ ]:
# === Cell 3: re-run training with callbacks ===

# if you want to resume from a prior 'last' checkpoint, uncomment:
# model = tf.   keras.models.load_model(
#     'braintumnet_last.h5',
#     custom_objects={'DiceLoss': DiceLoss, 'dice_score': dice_score}
# )

# Train (or resume) – here we train for 10 epochs total
    # now fit exactly as before, with your callbacks…
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=50,
    callbacks=callbacks,
    verbose=2
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50


I0000 00:00:1754075756.547713      94 service.cc:148] XLA service 0x7a05840027f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754075756.548592      94 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1754075758.007497      94 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-08-01 19:16:08.652976: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[128,96,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[8,96,256,256]{3,2,1,0}, f32[8,128,256,256]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]} is taking a while...
2025-08-01 19:16:09.878546: E external/l


Epoch 1: val_segmentation_dice_score improved from -inf to 0.02990, saving model to braintumnet_best.h5

Epoch 1: saving model to braintumnet_last.h5
253/253 - 87s - 343ms/step - classification_accuracy: 0.5830 - classification_loss: 0.9147 - loss: 1.7396 - segmentation_dice_score: 0.0824 - segmentation_loss: 1.0996 - val_classification_accuracy: 0.4515 - val_classification_loss: 1.6673 - val_loss: 2.2532 - val_segmentation_dice_score: 0.0299 - val_segmentation_loss: 1.1047 - learning_rate: 1.0000e-04
Epoch 2/50

Epoch 2: val_segmentation_dice_score improved from 0.02990 to 0.13851, saving model to braintumnet_best.h5

Epoch 2: saving model to braintumnet_last.h5
253/253 - 32s - 125ms/step - classification_accuracy: 0.6515 - classification_loss: 0.7910 - loss: 1.4176 - segmentation_dice_score: 0.2198 - segmentation_loss: 0.8637 - val_classification_accuracy: 0.4515 - val_classification_loss: 1.4046 - val_loss: 1.9369 - val_segmentation_dice_score: 0.1385 - val_segmentation_loss: 0.964

****to resume later****

In [ ]:
#to resume later


# rebuild & compile
model = build_braintumnet()
model.compile(...)

# load last checkpoint
model = tf.keras.models.load_model(
    'braintumnet_last.h5',
    custom_objects={'DiceLoss': DiceLoss, 'dice_score': dice_score}
)

# continue training up to, say, 20 epochs total
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    initial_epoch=10,
    callbacks=callbacks,
    verbose=2
)


In [ ]:
best_model = tf.keras.models.load_model(
    'braintumnet_best.h5',
    custom_objects={'DiceLoss': DiceLoss, 'dice_score': dice_score}
)
# then do: preds_seg, preds_cls = best_model.predict(my_test_images)


In [ ]:
# 1) Your TEST-set dirs
test_img_dir  = '/kaggle/input/figs-braintumor-segmentation-classification-sorted/restructured_dataset/segmentation_task/test/images'
test_mask_dir = '/kaggle/input/figs-braintumor-segmentation-classification-sorted/restructured_dataset/segmentation_task/test/masks'
test_cls_dir  = '/kaggle/input/figs-braintumor-segmentation-classification-sorted/restructured_dataset/classification_task/test'

# 2) Build the test DataFrame
class_map = {'glioma':0, 'meningioma':1, 'pituitary':2}
records = []

for cls_name, cls_idx in class_map.items():
    cls_folder = os.path.join(test_cls_dir, cls_name)
    if not os.path.isdir(cls_folder):
        continue

    for fname in os.listdir(cls_folder):
        if not fname.lower().endswith('.jpg'):
            continue

        # fname is e.g. "2948.jpg"
        base, _   = os.path.splitext(fname)
        seg_fname = f"{cls_name}_{base}.jpg"    # e.g. "glioma_2948.jpg"
        mask_fname= f"{cls_name}_{base}.png"

        img_path  = os.path.join(test_img_dir, seg_fname)
        mask_path = os.path.join(test_mask_dir, mask_fname)

        # only keep if both exist
        if not (os.path.exists(img_path) and os.path.exists(mask_path)):
            print(f"[!] skipping missing pair: {seg_fname} / {mask_fname}")
            continue

        # we’ll feed the generator seg_fname + label
        records.append({
            'filename': seg_fname,
            'label':    cls_idx
        })

test_df = pd.DataFrame(records).reset_index(drop=True)
print(f"Found {len(test_df)} test samples.")

# 3) Instantiate the TEST generator exactly as before
test_gen = MultiTaskDataGenerator(
    test_df,
    image_dir=test_img_dir,
    mask_dir=test_mask_dir,
    batch_size=8
)

# quick sanity
print("Generator length (batches):", len(test_gen))

In [ ]:
# 1) Load test DataFrame
test_df = load_brisc_data(DATA_DIR, split='test')
# If you have test_df:
test_gen = MultiTaskDataGenerator(test_df, batch_size=8, img_size=(256,256), augment = False)
print(f"Test samples: {len(test_df)}")

# 2) Build arrays
X         = []
Y_seg_true= []
y_cls_true= []

for _, row in test_df.iterrows():
    # grayscale read
    img  = cv2.imread(row['image_path'], cv2.IMREAD_GRAYSCALE)
    msk  = cv2.imread(row['mask_path'],  cv2.IMREAD_GRAYSCALE)
    if img is None or msk is None:
        continue

    # resize & normalize
    img = cv2.resize(img, (256,256)) / 255.0
    msk = cv2.resize(msk, (256,256)) / 255.0

    # add channel dim
    X.append(img.astype(np.float32)[...,None])
    Y_seg_true.append(msk.astype(np.float32)[...,None])
    y_cls_true.append(row['label'])

X          = np.stack(X, axis=0)
Y_seg_true = np.stack(Y_seg_true, axis=0)
y_cls_true = np.array(y_cls_true, dtype=int)

print(f"  → Loaded X: {X.shape}, Y_seg: {Y_seg_true.shape}, y_cls: {y_cls_true.shape}")

# 3) Load your best model
best_model = tf.keras.models.load_model(
    'braintumnet_best.h5',
    custom_objects={'DiceLoss': DiceLoss, 'dice_score': dice_score}
)

# 4) Predict once
preds_seg, preds_cls = best_model.predict(X, verbose=1)

# ── Segmentation metrics ─────────────────────────────────────────────────────
# binarize & compute per‐image Dice
dice_scores = []
for true, pred in zip(Y_seg_true, preds_seg):
    p_bin = (pred > 0.5).astype(np.float32)
    inter = np.sum(true * p_bin)
    d = (2*inter + 1e-6) / (np.sum(true) + np.sum(p_bin) + 1e-6)
    dice_scores.append(d)

print(f"\nSegmentation Mean Dice on test set: {np.mean(dice_scores):.4f}")

# ── Classification metrics ────────────────────────────────────────────────────
y_pred = np.argmax(preds_cls, axis=1)

print("\nClassification report:")
print(classification_report(
    y_cls_true,
    y_pred,
    labels=[0,1,2],
    target_names=['glioma','meningioma','pituitary']
))

print("Confusion matrix:")
print(confusion_matrix(y_cls_true, y_pred, labels=[0,1,2]))

In [ ]:
#code for inference on a few test images


# 1) Where your test images live
test_img_dir  = '/kaggle/input/figs-braintumor-segmentation-classification-sorted/restructured_dataset/segmentation_task/test/images'

# 2) Pick a few filenames you want to run
sample_files = [
    'glioma_1841.jpg',
    'meningioma_102.jpg',
    'pituitary_1071.jpg'
]

# 3) Reverse map from class‐index back to name
class_map_inv = {0:'glioma', 1:'meningioma', 2:'pituitary'}

for fname in sample_files:
    img_path = os.path.join(test_img_dir, fname)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Could not read {fname}, skipping.")
        continue

    # 4) Preprocess
    img_resized = cv2.resize(img, (256,256)) / 255.0
    inp = img_resized.astype(np.float32)[..., np.newaxis]       # shape (256,256,1)
    inp_batch = np.expand_dims(inp, 0)                          # shape (1,256,256,1)

    # 5) Predict
    seg_pred, cls_pred = best_model.predict(inp_batch, verbose=0)
    seg_mask = (seg_pred[0,...,0] > 0.5).astype(np.uint8)       # threshold at 0.5

    cls_idx  = np.argmax(cls_pred[0])
    cls_conf = cls_pred[0, cls_idx]
    cls_name = class_map_inv[cls_idx]

    print(f"{fname}  →  Predicted class: {cls_name} ({cls_conf:.3f})")

    # 6) Visualize
    plt.figure(figsize=(8,4))
    plt.subplot(1,2,1)
    plt.imshow(img_resized, cmap='gray')
    plt.title('Input')
    plt.axis('off')

    plt.subplot(1,2,2)
    plt.imshow(seg_mask, cmap='gray')
    plt.title('Predicted Mask')
    plt.axis('off')

    plt.show()


In [ ]:
!rm -rf /kaggle/working/*